# Imports 

In [ ]:
from PIL import Image
import numpy as np
from skimage import measure
import time
import gc
from matplotlib import pyplot as plt

# Code excution

In [ ]:
# Get the data
mask = Image.open("/root/workspace/data/prediction_15-02.png")
maskArray = np.array(mask)
maskArray = maskArray[:,:,0]/3 + maskArray[:,:,1]/3 + maskArray[:,:,2]/3

In [ ]:
# Split each the components
FN = np.copy(maskArray)
np.putmask(FN,FN!=85,0)

FP = np.copy(maskArray)
np.putmask(FP,FP!=170,0)

TP = np.copy(maskArray)
np.putmask(TP,TP!=255,0)

all_white = maskArray
np.putmask(all_white,all_white!=0,255)

In [ ]:
# Find the connected parts
all_labels = measure.label(all_white)

In [ ]:
# Free memory
del maskArray
del all_white
gc.collect()

In [ ]:
# First version to find the connected parts with TP
#start = time.time()
#test = np.copy(all_labels)
#for i in range(1,all_labels.max()+1):
#    indexes = np.where(test==i)
#    for j in range(len(indexes[0])):
#        if(TP[indexes[0][j]][indexes[1][j]]!=0):
#            np.putmask(test,test==i,255)
#            break
#np.putmask(test,test!=255,0)
#end = time.time()
#print(end - start)                  

In [ ]:
# Remove the FP and FN if they are onnected to a TP
connected_labels = np.copy(all_labels)
all_connected = np.copy(all_labels)
TP_copy = np.copy(TP)

# Number of different connected component
max_val = connected_labels.max()

np.putmask(TP_copy,TP_copy!=0,connected_labels.max())

# Here is a trick : We add the number of parts to the component with TP, 
# thus we won't have the same labels for a already existing label and a new one
connected_labels = connected_labels + TP_copy
connected_labels = np.unique(connected_labels)

# We retrieve the parts with TP
indexes = np.where(connected_labels>max_val)
connected_labels = np.delete(connected_labels, np.s_[:indexes[0][0]], axis=0)

# And we put them in white
for i in connected_labels:
    np.putmask(all_connected,all_connected==i-max_val,-1)
np.putmask(all_connected,all_connected>0,0)
np.putmask(all_connected,all_connected==-1,255)

In [ ]:
# Free memory
del connected_labels
del indexes
del all_labels
gc.collect()

In [ ]:
# Put the FN and FP on the result
all_data = np.copy(all_connected)
all_data += (FN + FP).astype(np.int64)
np.putmask(all_data, all_data>255, 255)

In [ ]:
# Free memory
del all_connected
gc.collect()

In [ ]:
# Display the result
plt.figure(figsize=(50,50))
plt.subplot(131)
plt.imshow(all_data, cmap='nipy_spectral')# Free memory
plt.show()

In [ ]:
# Retrieve the new FN, FP and TP
FN = np.copy(all_data)
np.putmask(FN,FN!=85,0)

FP = np.copy(all_data)
np.putmask(FP,FP!=170,0)

TP = np.copy(all_data)
np.putmask(TP,TP!=255,0)

In [ ]:
# Free memory
del all_data
gc.collect()

In [ ]:
# Find the new connected components
FN_labels = measure.label(FN)
FP_labels = measure.label(FP)
TP_labels = measure.label(TP)

In [ ]:
# Get the number of TP FP and FN
FPn = FP_labels.max()
FNn = FN_labels.max()
TPn = TP_labels.max()
print("TP : ",TPn, " FP : ", FPn , " FN : ", FNn)

In [ ]:
# Free memory
del FN
del FP
del TP
del FN_labels
del FP_labels
del TP_labels
gc.collect()

In [ ]:
# Calculate the Fscore
Fscore = (2*TPn)/(2*TPn+FPn+FNn)
print("Fscore : ", Fscore)